[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/experiments/small_loop.ipynb)

# THE SMALL LOOP ENVIRONMENT

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from ipywidgets import Button, HBox,VBox, Output
from IPython.display import display

## Define the actions

In [2]:
FORWARD = 0
TURN_LEFT = 1
TURN_RIGHT = 2
FEEL_FRONT = 3
FEEL_LEFT = 4
FEEL_RIGHT = 5

## Define the environment 

In [371]:
colors = ['white', 'green', 'yellow', 'yellow', 'red']
colors = ["#BE9CC7", '#9F8DC3', '#E3BAD5', '#5C5792', "red"]
agent_color = "#211A44"
colors = ["#eec9c9", '#e8b6b6', '#f0f1f3', '#8b94a3', "red"]
agent_color = "#727c8f"
colors = ["#cccccc", '#a5a5a5', '#f2f2f2', '#7f7f7f', "red"]
agent_color = "#4392F1"
size = 200

empty_color = '#D6D6D6'
wall_color = '#5C946E'
feel_empty_color = '#FAE2DB'
feel_wall_color = '#535865'
bump_color = "#F93943"  # "#BB342F"  # "#F62DAE"
agent_color = "#1976D2"
colors = [empty_color, wall_color, feel_empty_color, feel_wall_color, bump_color]
#colors = ["#D6D6D6", '#5C946E', '#FAE2DB', '#535865', "#F93943"]


In [372]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
FEELING_EMPTY = 2
FEELING_WALL = 3
BUMPING = 4

class SmallLoop():
    def __init__(self, poX, poY, direction):
        self.grid = np.array([
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            [1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
            [1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
            [1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
            [1, 0, 0, 0, 0, 1, 1, 1, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
            [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
            [1, 0, 0, 1, 1, 1, 1, 1, 0, 1],
            [1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.poX = poX
        self.poY = poY
        self.direction = direction
        self.cmap = ListedColormap(colors)
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5, 4.5], self.cmap.N)

    def outcome(self, action):
        # print('before:', self.agent_position.strPosition(), action_dcit[action])
        self.maze[:,:] = self.grid
        result = 0
        
        if action == FORWARD:  # move forward
            # print('the action is move forward')
            # print(str(self.position.pointX)+': '+ str(self.position.pointY)+ ' ' +self.direction, action)
        
            if self.direction == LEFT:
                if self.maze[self.poX][self.poY - 1] == 0:
                    self.poY -= 1
                else:
                    self.maze[self.poX][self.poY - 1] = BUMPING
                    result = 1
            elif self.direction == DOWN:
                if self.maze[self.poX + 1][self.poY] == 0:
                    self.poX += 1
                else:
                    self.maze[self.poX + 1][self.poY] = BUMPING
                    result = 1
            elif self.direction == RIGHT:
                if self.maze[self.poX][self.poY + 1] == 0:
                    self.poY += 1
                else:
                    self.maze[self.poX][self.poY + 1] = BUMPING
                    result = 1
            elif self.direction == UP:
                if self.maze[self.poX - 1][self.poY] == 0:
                    self.poX -= 1
                else:
                    self.maze[self.poX - 1][self.poY] = BUMPING
                    result = 1
            # print(str(self.position.pointX)+': '+ str(self.position.pointY)+ ' ' +self.direction, action)
        elif action == TURN_RIGHT:
            if self.direction == LEFT:
                self.direction = UP
            elif self.direction == DOWN:
                self.direction = LEFT
            elif self.direction == RIGHT:
                self.direction = DOWN
            elif self.direction == UP:
                self.direction = RIGHT
        elif action == TURN_LEFT:
            if self.direction == LEFT:
                self.direction = DOWN
            elif self.direction == DOWN:
                self.direction = RIGHT
            elif self.direction == RIGHT:
                self.direction = UP
            elif self.direction == UP:
                self.direction = LEFT
        elif action == FEEL_FRONT:
            if self.direction == LEFT:
                if self.maze[self.poX][self.poY - 1] == 0:
                    self.maze[self.poX][self.poY - 1] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX][self.poY - 1] = FEELING_WALL
            elif self.direction == DOWN:
                if self.maze[self.poX + 1][self.poY] == 0:
                    self.maze[self.poX + 1][self.poY] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX + 1][self.poY] = FEELING_WALL
            elif self.direction == RIGHT:
                if self.maze[self.poX][self.poY + 1] == 0:
                    self.maze[self.poX][self.poY + 1] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX][self.poY + 1] = FEELING_WALL
            elif self.direction == UP:
                if self.maze[self.poX - 1][self.poY] == 0:
                    self.maze[self.poX - 1][self.poY] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX - 1][self.poY] = FEELING_WALL
        elif action == FEEL_LEFT:
            if self.direction == LEFT:
                if self.maze[self.poX + 1][self.poY] == 0:
                    self.maze[self.poX + 1][self.poY] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX + 1][self.poY] = FEELING_WALL
            elif self.direction == DOWN:
                if self.maze[self.poX][self.poY + 1] == 0:
                    self.maze[self.poX][self.poY + 1] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX][self.poY + 1] = FEELING_WALL
            elif self.direction == RIGHT:
                if self.maze[self.poX - 1][self.poY] == 0:
                    self.maze[self.poX - 1][self.poY] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX - 1][self.poY] = FEELING_WALL
            elif self.direction == UP:
                if self.maze[self.poX][self.poY - 1] == 0:
                    self.maze[self.poX][self.poY - 1] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX][self.poY - 1] = FEELING_WALL
        elif action == FEEL_RIGHT:
            if self.direction == LEFT:
                if self.maze[self.poX - 1][self.poY] == 0:
                    self.maze[self.poX - 1][self.poY] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX - 1][self.poY] = FEELING_WALL
            elif self.direction == DOWN:
                if self.maze[self.poX][self.poY - 1] == 0:
                    self.maze[self.poX][self.poY - 1] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX][self.poY - 1] = FEELING_WALL
            elif self.direction == RIGHT:
                if self.maze[self.poX + 1][self.poY] == 0:
                    self.maze[self.poX + 1][self.poY] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX + 1][self.poY] = FEELING_WALL
            elif self.direction == UP:
                if self.maze[self.poX][self.poY + 1] == 0:
                    self.maze[self.poX][self.poY + 1] = FEELING_EMPTY
                else:
                    result = 1
                    self.maze[self.poX][self.poY + 1] = FEELING_WALL
        print(f"Line: {self.poX}, Column: {self.poY}, direction: {self.direction}")
        # return self.position,

        return result  
    
    def display(self):
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            # ax.set_xticks([])
            # ax.set_yticks([])
            # ax.axis('off')
            # ax.imshow(self.maze, cmap='Greens', vmin=0, vmax=2)
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            if self.direction == LEFT:
                # Y is column and X is line
                plt.scatter(self.poY, self.poX, s=size, marker='<', c=agent_color)
            elif self.direction == DOWN:
                plt.scatter(self.poY, self.poX, s=size, marker='v', c=agent_color)
            elif self.direction == RIGHT:
                plt.scatter(self.poY, self.poX, s=size, marker='>', c=agent_color)
            else: # UP
                plt.scatter(self.poY, self.poX, s=size, marker='^', c=agent_color)
            plt.show()

    def save(self, step):
        """
        save the display as png file
        """
        fig, ax = plt.subplots()
        ax.set_xticks([])
        ax.set_yticks([])
        ax.axis('off')
        ax.imshow(self.maze, cmap='Greens', vmin=0, vmax=2)
        ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
        if self.direction == LEFT:
            # Y is column and X is line
            plt.scatter(self.poY, self.poX, s=size, marker='<', c=agent_color)
        elif self.direction == DOWN:
            plt.scatter(self.poY, self.poX, s=size, marker='v', c=agent_color)
        elif self.direction == RIGHT:
            plt.scatter(self.poY, self.poX, s=size, marker='>', c=agent_color)
        else: # UP
            plt.scatter(self.poY, self.poX, s=size, marker='^', c=agent_color)

        # Add number in plot
        ax.text(4, 0, f"{step:>3}", fontsize=12, color='White')
        plt.savefig(f"{save_dir}/{step:03}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)

## Display the environment

In [373]:
small_loop = SmallLoop(1, 1, 0)

Create the control buttons

In [374]:
def forward(change):
    small_loop.outcome(FORWARD)
    small_loop.display()

def turn_left(change):
    small_loop.outcome(TURN_LEFT)
    small_loop.display()

def turn_right(change):
    small_loop.outcome(TURN_RIGHT)
    small_loop.display()

def feel_left(change):
    small_loop.outcome(FEEL_LEFT)
    small_loop.display()

def feel_right(change):
    small_loop.outcome(FEEL_RIGHT)
    small_loop.display()

def feel_front(change):
    small_loop.outcome(FEEL_FRONT)
    small_loop.display()

# Create Start and Stop buttons
forward_button = Button(description="Forward")
left_button = Button(description="Turn left")
right_button = Button(description="Turn right")
feel_left_button = Button(description="Feel left")
feel_front_button = Button(description="Feel Front")
feel_right_button = Button(description="Feel right")

# Bind button clicks to functions
forward_button.on_click(forward)
left_button.on_click(turn_left)
right_button.on_click(turn_right)
feel_left_button.on_click(feel_left)
feel_right_button.on_click(feel_right)
feel_front_button.on_click(feel_front)

# Create the control bar
buttons = HBox([forward_button, left_button, right_button, feel_front_button, feel_left_button, feel_right_button])

In [375]:
out = Output()
small_loop.display()
display(out)
display(buttons)

Output()

## Enact actions

Use `Ctrl+Enter` to run the cell below and stay on it.

In [11]:
result = small_loop.outcome(FORWARD) # 0
small_loop.display()
print(f"outcome: {result}") 

Line: 4, Column: 1, direction: 1
outcome: 0


In [109]:
result = small_loop.outcome(TURN_LEFT) # 1
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 1
outcome: 0


In [117]:
result = small_loop.outcome(TURN_RIGHT) # 2
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 3
outcome: 0


In [108]:
result = small_loop.outcome(FEEL_FRONT) # 3
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1


In [105]:
result = small_loop.outcome(FEEL_LEFT) # 4
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 0


In [107]:
result = small_loop.outcome(FEEL_RIGHT) # 5
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1
